In [2]:
sys.path.append('..')
from config import Config

import json
import numpy as np # Для работы с данными 
import sys
import os

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами

In [3]:
JSON_DATA_DIR = '../../Satellites'

In [17]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
textClasses = ['', '']


# открываем файл с id эссе и ответами
with open(JSON_DATA_DIR + '/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    with open(JSON_DATA_DIR + f'/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if elem['answer'] == False:
        textClasses[0] += text
        textClasses[0] += '#'
      else:
        textClasses[1] += text
        textClasses[1] += '#'

In [18]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
texts = []

# открываем файл с id эссе и ответами
with open(JSON_DATA_DIR + '/test/test_task.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    with open(JSON_DATA_DIR + f'/test/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      texts.append([text])

In [19]:
texts_false = textClasses[0].split("#")
texts_true = textClasses[1].split("#")

trainText = []
trainText.append(' '.join(texts_false))
trainText.append(' '.join(texts_true))

In [20]:
tokenizer = Tokenizer(
    filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    oov_token='unknown',
    char_level=False)

tokenizer.fit_on_texts(trainText)

In [21]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClassesTest(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wordIndexes[0], xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [22]:
xLen = 300

In [23]:
essays = []
yTest = []

for i in range(len(texts)):
    testTextArray = tokenizer.texts_to_sequences(texts[i][0])
    xTestId, _ = createSetsMultiClassesTest(testTextArray, xLen, xLen)
    essays.append(xTestId)


In [24]:
print(xTestId[0])

IndexError: index 0 is out of bounds for axis 0 with size 0